PowerSGD는 DDP에서 구현되어있습니다.
https://medium.com/pytorch/accelerating-pytorch-ddp-by-10x-with-powersgd-585aef12881d 파이토치공식 블로그 튜토리얼 참고한 자료

강의에서 우리가 다뤘던건 Deep gradient compression (DGC) 이었습니다. Gradient 를 sparse하게 만들어서 통신에서 이동하는 gradient 양 자체를 줄여서 통신 오버헤드를 줄이는 방법이었습니다. 

분산 학습의 통신 효율성을 개선하기 위한 대중적인 접근 방식입니다. 그러나 DGC는 torch 에서 라이브러리고 구현이 되어있지 않고, 그래서 만약에 이걸 쓰고싶다면 low-level 로 새롭게 구현해야합니다. PyTorch는 최신 알고리즘 중 하나인 PowerSGD를 DDP communication hook으로 지원하고 있습니다. DDP 통신 훅은 PyTorch 1.10에서 안정(stable) 기능으로 출시되었으며, NCCL (nvidia GPU 용), Gloo (CPU 용), MPI를 포함한 여러 통신 백엔드와 함께 작동할 수 있다는 점에서 DDP와 비슷하게/ 같은 모듈을 사용해서 만들어져있습니다. 

![DDP](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*o_d6rHbPRyUOf1TAEzaHRQ.png)

그래서 본 튜토리얼에서는 powerSGD 라는 또다른 gradient compression 기법을 다룹니다.

# PowerSGD

PowerSGD는 distributed data-parallel(DDP) 훈련에서 통신 오버헤드를 줄이기 위해 사용되는 그래디언트 압축 기법입니다. 이 방법은 그래디언트 행렬을 저랭크 근사로 표현하여 전송해야 하는 데이터의 양을 줄입니다. 

![PowerSGD](./img/powersgdd.png)

PowerSGD 그래디언트 압축 알고리즘은 (Thijs Vogels 등이) NeurIPS'19 및 NeurIPS'20에 발표했습니다. PowerSGD는 벡터가 아닌(non-vector) 각 그래디언트 텐서를 행렬로 보고, 파워 이터레이션(power iteration, 멱반복법)에 기반하여 압축합니다. 이는 MxN 형태의 텐서를 각각 MxR과 RxN 형태의 저차원(low-rank) 텐서 두 개로 인수분해합니다 (여기서 R은 구성 가능한 파라미터입니다).

PowerSGD 압축기(compressor)는 계산적으로 훨씬 가벼우며(lightweight) 학습이 진행됨에 따라 점진적으로 행렬 근사(approximation)를 개선할 수 있습니다. PowerSGD 논문의 인용문입니다:

> “With gradients shaped as in POWERSGD, computing the SVD of a stochastic gradient takes 673ms, the equivalent of computing 6 mini-batch gradients. In contrast, one full step of rank-2 POWERSGD, including communication between 16 workers, takes only 105ms.”

나중에 다른것도 라이브러리 사용하는 것들도 얘기하겠지만 (deepspeed) powerSGD가 유용한 이유는 native pytorch로 작동한다는 점입니다. optimizer, model 전부 torch로 써져있는 상태로 써먹을 수 있기 때문에 쉽게 적용 가능합니다. deepspeed같은걸 쓰면 하드웨어를 타고, optimizer-model 도 그에 맞게 다시 만들어야하거나 추가적인 내용이 필요합니다. 예를들어 deepspeed를 이용해서 1bit DGC을 한다면 그걸 반영할때는 다시 32bit FP tensor 로 만들어주고.. 하는 내용이 있어야합니다. 

DDP에 PowerSGD를 적용하려면, DDP 모델이 생성된 후 아래와 같이 PowerSGD 통신 훅을 등록하기만 하면 됩니다. 

## DDP Communication Hooks

Hook이라는건 일반적으로 어떤 이벤트가 발생했을 때 호출되는 콜백 함수를 의미합니다. DDP에서 communication hook은 DDP를 적용한 상태에서 오리지날 DDP적용 방식에 더해 사용자가 정의한 콜백 함수를 통해 통신 과정을 커스터마이징하는 목적을 갖고 있습니다.

그래서 뭐 deep gradient compression (DGC) 같은걸 직접 구현해서 쓸 수도 있겠죠

`DistributedDataParallel`에 통신 훅을 등록하려면 `register_comm_hook()`을 사용해서 등록해두면 나중에 `loss.backward()`가 호출될 때마다 해당 훅이 실행됩니다.

```python
from torch.distributed.algorithms.ddp_comm_hooks.default_hooks import (
    allreduce_hook
)

ddp_model.register_comm_hook(ddp_model.process_group, bf16_compress_hook)
```

이런식으로 ddp 로 선언된 모델에서 bf16_compress_hook 이라는 함수를 실행시킬 수 있습니다. `loss.backward()`가 실행될때 DDP의 기본 그래디언트 동기화 (all-reduce) 대신 실행됩니다. 
그래서 훅을 넣을려면 allreduce를 대체하는 함수 (powerSGD) 혹은 DGC 같은걸 직접 구현해서 넣어줘야하며, 그게 아니라면 allreduce 역시도 register 해야합니다. 


```python
from torch.distributed.algorithms.ddp_comm_hooks.default_hooks import (
    bf16_compress_wrapper, 
    bf16_compress_hook,
)

ddp_model.register_comm_hook(ddp_model.process_group, bf16_compress_wrapper(allreduce_hook))
```

이런식으로 


...

아무튼 다시 powerSGD로 돌아와서, PowerSGD 통신 훅을 만들어서 등록하면 됩니다. 

`torch.distributed.algorithms.ddp_comm_hooks.powerSGD_hook.powerSGD_hook`

(state: PowerSGDState, bucket: GradBucket) -> Future[Tensor]

```python
import torch.distributed.algorithms.ddp_comm_hooks.powerSGD_hook as powerSGD

# DDP 모델 생성
ddp_model = DDP(model, device_ids=[local_rank])

# PowerSGD 상태(state) 초기화
state = powerSGD.PowerSGDState(
    process_group=ddp_model.process_group,
    matrix_approximation_rank=1, # rank 1로 압축
    start_powerSGD_iter=1000 # 1000 스텝 이후 압축 시작
)

# DDP 모델에 훅(hook) 등록
ddp_model.register_comm_hook(state, powerSGD.powerSGD_hook)

# ... 이후 평소처럼 학습 루프 실행 ...
```

이를 평가하기 위해서 일단 `nvidia-smi dmon -s put` 명령어로 보내고 받는  대역폭을 측정할 수있습니다. 

그 다음 `mnist_powersgd_torchrun.py`를 실행해봅시다.